## Travel time analysis

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np

### Data reading

In [55]:
# Read in travel time results and SourceSink data
scenarios = range(0,5)
results = {}
for s in scenarios:
    results[s] = pd.read_csv(f"../experiments/results_scenario_{s}.csv", index_col=0)
    results[s].drop("VehicleID", axis="columns", inplace=True)
source_sinks = pd.read_csv(f"../experiments/source_data.csv", index_col=0)

In [58]:
display(results[4].head(8))
display(source_sinks.head(8))

,Travel_Time,Startpoint,Endpoint
0,14,SourceSink13033,13022
1,18,SourceSink13022,13023
2,14,SourceSink10000,12254
3,6,SourceSink13034,13040
4,14,SourceSink13022,13033
5,6,SourceSink13034,13040
6,6,SourceSink12750,12756
7,20,SourceSink13034,13033


,Road,Coordinates,Startpoint,Endpoint
10000,N1,"(90.443333, 23.7060278)",10000,10000
11302,N1,"(92.298083, 20.8629167)",11302,11302
11303,N2,"(90.5214438, 23.7059167)",11303,11303
12253,N2,"(92.0176382, 25.1570556)",12253,12253
12254,N105,"(90.5466108, 23.6904163)",12254,12254
12354,N105,"(90.3582222, 23.9895274)",12354,12354
12355,N102,"(91.1181938, 23.4789716)",12355,12355
12541,N102,"(91.1146667, 24.0506111)",12541,12541


In [57]:
source_sinks['Startpoint']= source_sinks.index
source_sinks['Endpoint']= source_sinks.index

In [59]:
scenario0data = results[0]
scenario1data = results[1]
scenario2data = results[2]
scenario3data = results[3]
scenario4data = results[4]

In [73]:
scenario1data.head()

,Travel_Time,Startpoint,Endpoint
0,9,13022,13012
1,6,13040,13034
2,13,13033,13012
3,6,12756,12750
4,6,12750,12756


In [74]:
source_sinks.head()

,Road,Coordinates,Startpoint,Endpoint
10000,N1,"(90.443333, 23.7060278)",10000,10000
11302,N1,"(92.298083, 20.8629167)",11302,11302
11303,N2,"(90.5214438, 23.7059167)",11303,11303
12253,N2,"(92.0176382, 25.1570556)",12253,12253
12254,N105,"(90.5466108, 23.6904163)",12254,12254


In [104]:
hoi = pd.merge(scenario1data, source_sinks, on = ['Startpoint'])

In [105]:
hoi.head()

,Travel_Time,Startpoint,Endpoint_x,Road,Coordinates,Endpoint_y
0,9,13022,13012,N209,"(91.868166, 24.8859441)",13022
1,18,13022,13023,N209,"(91.868166, 24.8859441)",13022
2,9,13022,13012,N209,"(91.868166, 24.8859441)",13022
3,55,13022,12871,N209,"(91.868166, 24.8859441)",13022
4,14,13022,13033,N209,"(91.868166, 24.8859441)",13022


In [107]:
hoi =hoi.rename(columns={'Road':'StartRoad', 'Endpoint_x':'Endpoint'})

In [102]:
hoi.head()
hoi = pd.merge(scenario1data, source_sinks, on = ['Endpoint'])

,Travel_Time,Startpoint_x,Endpoint,Road,Coordinates,Startpoint_y
0,9,13022,13012,N209,"(91.8753604, 24.8775549)",13012
1,13,13033,13012,N209,"(91.8753604, 24.8775549)",13012
2,31,13040,13012,N209,"(91.8753604, 24.8775549)",13012
3,9,13022,13012,N209,"(91.8753604, 24.8775549)",13012
4,54,12871,13012,N209,"(91.8753604, 24.8775549)",13012


In [89]:
scenario1data['Startpoint'] = scenario1data['Startpoint'].str.replace('SourceSink','')
scenario1data['Startpoint'] = scenario1data['Startpoint'].fillna(0).astype(int)

In [64]:
scenario1data['Startroad','Endroad']

KeyError: ('Startroad', 'Endroad')

### Global average travel time

In [4]:
#calculating 95% confidence intervals of the economic losses
confidence = 0.95

average = {}
low_bound = {}
high_bound = {}
interval_range = {}
economic_interval = {}

for i in scenarios:
    results_list = results[i]["Travel_Time"].tolist()
    average[i] = np.mean(results_list)
    low_bound[i], high_bound[i] = st.norm.interval(alpha=confidence, loc=average[i], scale=st.sem(results_list))
    interval_range[i] = high_bound[i] - low_bound[i]
    economic_interval[i] = interval_range[i] / average[i] * 100

    print(f'Average travel time (95% confidence interval) for scenario {i}: {average[i]:.3f} ({low_bound[i]:.3f}, {high_bound[i]:.3f}), economic interval: {economic_interval[i]:.3f}%')

df = pd.DataFrame({
    "Average (min)": average,
    "Low bound (min)": low_bound,
    "High bound (min)": high_bound,
    "Interval range (min)": interval_range,
    "Economic interval (%)": economic_interval})
df.to_csv("../results/travel_times.csv", index_label="Scenario")
df

Average travel time (95% confidence interval) for scenario 0: 322.690 (318.780, 326.600), economic interval: 2.424%
Average travel time (95% confidence interval) for scenario 1: 325.522 (321.554, 329.490), economic interval: 2.438%
Average travel time (95% confidence interval) for scenario 2: 357.762 (353.048, 362.475), economic interval: 2.635%
Average travel time (95% confidence interval) for scenario 3: 392.587 (387.434, 397.740), economic interval: 2.625%
Average travel time (95% confidence interval) for scenario 4: 438.301 (429.359, 447.243), economic interval: 4.080%


,Average (min),Low bound (min),High bound (min),Interval range (min),Economic interval (%)
0,322.690026,318.779610,326.600442,7.820832,2.423636
1,325.521909,321.553621,329.490197,7.936576,2.438108
2,357.761645,353.048441,362.474850,9.426409,2.634829
3,392.586686,387.433747,397.739624,10.305877,2.625121
4,438.301127,429.359157,447.243098,17.883942,4.080287


### Average travel time per route